# Explorar y descargar datasets y modelos


Comencemos explorando el repositorio de conjuntos de datos y modelos.

Puedes hacerlo en las diferentes capas de accesibilidad de EOTDL: la interfaz de usuario, la API, la interfaz de línea de comandos (CLI) y la biblioteca de Python.


## La interfaz de usuario


La forma más sencilla de empezar a usar EOTDL es explorando la interfaz de usuario: [https://eotdl.com/](https://www.eotdl.com/). A través de ella, podrá:

- Explorar los conjuntos de datos y modelos disponibles en el repositorio (filtrando por nombre, etiquetas y preferencias).
- Editar la información de sus propios conjuntos de datos y modelos.
- Leer los tutoriales del blog.
- Leer la documentación.
- Encontrar enlaces útiles a otros recursos (GitHub, Discord, etc.).

![web](../img/eotdl.png)


## Interfaz de línea de comandos (CLI)


Aunque la interfaz de usuario es la forma más sencilla de empezar, no es la más práctica para trabajar con los conjuntos de datos y modelos. Para ello, recomendamos instalar la CLI.

Si ejecuta este notebook localmente, considere crear un entorno virtual antes de instalar la CLI para evitar conflictos con otros paquetes.

Con [uv](https://docs.astral.sh/uv/):


In [ ]:
# descomentar para instalar

# !uv add eotdl

Una vez instalado, puedes ejecutar la CLI con diferentes comandos.


In [ ]:
!eotdl --help

In [ ]:
!eotdl version

In [ ]:
!eotdl datasets --help

Se pueden explorar conjuntos de datos con el siguiente comando:


In [ ]:
!eotdl datasets list 

In [ ]:
!eotdl datasets list --help

In [ ]:
!eotdl datasets list -n eurosat

Como habrás adivinado, puedes preparar un conjunto de datos con el siguiente comando:


In [ ]:
!eotdl datasets get EuroSAT-RGB -v 1

La primera vez que ejecute el comando, se le pedirá que inicie sesión (lo que requerirá que cree una cuenta si aún no lo ha hecho). También puede iniciar sesión con el comando:


In [ ]:
!eotdl auth login

In [ ]:
!eotdl auth --help

In [ ]:
!eotdl datasets get --help

De forma predeterminada, los conjuntos de datos se almacenan en la carpeta `$HOME/.cache/eotdl/datasets` o en la ruta de la variable de entorno `EOTDL_DOWNLOAD_PATH`. Puede cambiar esto con el argumento `--path`.


In [ ]:
!eotdl datasets get EuroSAT-RGB -v 1 -p data

Puedes elegir una versión específica para descargar con el argumento `--version`. Si no especificas una versión, se descargará la más reciente.


In [ ]:
!eotdl datasets get EuroSAT-RGB -p data -v 1

Si intenta volver a preparar un conjunto de datos, la CLI generará un error. Puede forzar una nueva descarga con el argumento `--force`.


In [ ]:
!eotdl datasets get EuroSAT-RGB -p data -v 1 -f

De forma predeterminada, el comando `get` solo descargará los metadatos del conjunto de datos.


In [ ]:
!ls data/EuroSAT-RGB

El archivo `README.md` contiene información básica sobre el conjunto de datos que mostramos en la interfaz de usuario (autores, nombre, licencias, descripción, ...)


In [ ]:
!cat data/EuroSAT-RGB/README.md

Y `catalog.v[version].parquet` contiene los metadatos STAC para los conjuntos de datos, incluidos los enlaces a los datos sin procesar (activos).


In [ ]:
import geopandas as gpd

catalog = gpd.read_parquet("data/EuroSAT-RGB/catalog.v1.parquet")

catalog

In [ ]:
catalog.assets.values

Es posible preparar datos de la lista de activos del catálogo mediante la biblioteca (véase la siguiente sección), pero probablemente desee preparar todos los datos a la vez. Para ello, añada el argumento `--assets` al comando `get` mediante la CLI.


In [ ]:
!eotdl datasets get EuroSAT-RGB -p data -v 1 -f -a

In [ ]:
!ls data/EuroSAT-RGB

Trabajar con modelos es muy parecido en este punto.


In [ ]:
!eotdl models --help

In [ ]:
!eotdl models list

In [ ]:
!eotdl models list --help

In [ ]:
!eotdl models get RoadSegmentation -p data -a -f

In [ ]:
!eotdl models get --help

Exploraremos cómo ingerir conjuntos de datos y modelos en los próximos tutoriales.


## La librería de Python


Todo lo que hemos hecho hasta ahora con la CLI también se habilita mediante la biblioteca de Python. Al instalar la CLI, la biblioteca también se instala automáticamente.


In [ ]:
import eotdl

eotdl.__version__

In [ ]:
from eotdl.datasets import retrieve_datasets

datasets = retrieve_datasets()
len(datasets)

In [ ]:
retrieve_datasets("eurosat")

Con la biblioteca, tienes control total sobre los conjuntos de datos y modelos.


In [ ]:
[d for d in datasets if "eurosat" in d.lower()]

También puedes preparar conjuntos de datos, pero ahora tendrás que gestionar posibles errores.


In [ ]:
from eotdl.datasets import stage_dataset

stage_dataset("EuroSAT-RGB", force=True)

In [ ]:
stage_dataset("EuroSAT-RGB", version=1, force=True)

Es posible organizar activos particulares recuperando sus enlaces del catálogo.


In [ ]:
path = stage_dataset("EuroSAT-RGB", force=True, path="data", assets=True)

catalog = gpd.read_parquet(f"{path}/catalog.v1.parquet")

catalog

In [ ]:
from eotdl.datasets import stage_dataset_file

for _, row in catalog.iterrows():
    if row["id"] == "EuroSAT-RGB.zip":
        print(row["assets"]["asset"])
        stage_dataset_file(row["assets"]["asset"]["href"], "data")

In [ ]:
!ls data

De hecho, la CLI está construida sobre la biblioteca.


Y lo mismo para los modelos.


In [ ]:
from eotdl.models import retrieve_models

retrieve_models()

In [ ]:
from eotdl.models import stage_model

path = stage_model("RoadSegmentation", force=True, path="data", assets=True)
path

In [ ]:
import os

os.listdir(path)

## Oportunidades de discusión y contribución


No dudes en hacer preguntas ahora (en vivo o a través de Discord) y sugerir mejoras futuras.

- ¿Qué funciones te gustaría ver para explorar y descargar?
